In [ ]:
! pip install nbeats_pytorch

In [ ]:
! pip install ngboost

### import libraries

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from ngboost import NGBClassifier
from nbeats_pytorch.model import NBeatsNet
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

### dataset

In [ ]:
# 산림빅데이터거래소 - 농산물 소비인구 정보
# https://www.bigdata-forest.kr/product/FPL120101
data_nongsan = pd.read_csv('data/농산물_소비인구/fpl_tb_201217170010019000.csv', on_bad_lines='skip', delimiter='|',
                          names=['테이블순번','관심인구ID','정보획득일자','이용자성별코드','연령','연령그룹코드','이용자직업명',
                      '가구소득그룹명','결혼여부명','자녀여부명','주거형태명','이용자지역시도명','이용자지역구명','이용자지역동명',
                     '자동차그룹명','휴대전화제조사명','휴대전화가입회사명',
                                 '쇼핑몰구분','사이트메뉴명','사이트메뉴구분명','정보획득사이트URL','제품내용','1','2','3','4'], low_memory=False)
data_nongsan = data_nongsan.iloc[:, :-4]
data_nongsan

In [ ]:
# 산림 빅데이터 거래소 - 임산물 관심인구 정보
# https://www.bigdata-forest.kr/product/FPL090101

data_imsan = pd.read_csv('data/5thData/fpl_tb_201217164710015000.csv', encoding='cp949', on_bad_lines='skip', delimiter='|', names=['테이블순번','관심인구ID','추출단어명','임산물구분명','정보획득일자','이용자성별코드','연령그룹코드','이용자직업명',
                      '가구소득그룹명','결혼여부명','자녀여부명','주거형태명','이용자지역시도명','이용자지역구명','이용자지역동명',
                     '자동차그룹명','휴대전화제조사명','휴대전화가입회사명','정보획득사이트URL','정보획득사이트제목명','1','2','3','4'])
data_imsan = data_imsan.iloc[:, :-4]
data_imsan

In [ ]:
data_nongsan.info()

In [ ]:
data_imsan.info()

### 농산물 데이터 '제품내용' 컬럼에서 임산물명 추출

In [ ]:
# 임산물 리스트
imsan_type = ['대추', '호두', '머루', '복분자', '산마늘', '산딸기', '죽순', '돌배', '고사리', '표고버섯',
            '도라지', '감', '눈개승마', '곤드레', '꽃송이버섯', '밤', '도토리', '고려엉겅퀴', '송이버섯',
            '더덕', '참나물', '어수리', '두릅', '석이버섯', '석류', '싸리버섯', '잣', '능이버섯',
            '목이버섯', '복령버섯', '취나물', '개암', '다래', '고비', '원추리', '은행', '곰취',
            '들메나무순']

# 임산물명 추출해 '임산물명' 컬럼에 추가, 없으면 False
data_nongsan['임산물명'] = data_nongsan['제품내용'].apply(lambda x: next((imsan for imsan in imsan_type if imsan in x), 'None'))

# 체크
data_nongsan['임산물명']

In [ ]:
# 예외 처리 #############
# 밤고구마, 굿밤 -> None
# 감귤, 포만감, 과일보감, 감자 -> None
# 대추방울토마토 -> None
data_nongsan.loc[(data_nongsan['임산물명'] != 'None') &
                 (data_nongsan['제품내용'].str.contains('밤고구마|굿밤|감귤|포만감|보감|감자')), '임산물명'] = 'None'

# 곤드레, 감자탕 -> 같이 들어간 건 곤드레로..
data_nongsan.loc[(data_nongsan['임산물명'] == 'None') &
                 (data_nongsan['제품내용'].str.contains('곤드레')) &
                 (data_nongsan['제품내용'].str.contains('감자탕')), '임산물명'] = '곤드레'

# 홍시, 연시, 건시 -> 감
data_nongsan.loc[(data_nongsan['임산물명'] == 'None') &
                 (data_nongsan['제품내용'].str.contains('홍시|연시|건시')), '임산물명'] = '감'

In [ ]:
data_nongsan['임산물명'].value_counts()

![KakaoTalk_Photo_2024-11-14-14-06-49.png](attachment:b7b89ac7-d69f-48dc-b0f2-bbb951078582.png)

### 분류 간소화

In [ ]:
# 분류 기준 딕셔너리 생성
category_mapping = {
    '견과류': ['호두', '밤', '도토리', '잣', '개암', '은행'],
    '버섯류': ['표고버섯', '꽃송이버섯', '송이버섯', '석이버섯', '싸리버섯', '능이버섯', '목이버섯', '복령버섯'],
    '나물류': ['산마늘', '고사리', '눈개승마', '곤드레', '고려엉겅퀴', '참나물', '어수리', '두릅', '취나물', '고비', '원추리', '곰취', '들메나무순'],
    '열매과일류': ['대추', '머루', '복분자', '산딸기', '돌배', '감', '석류', '다래'],
    '뿌리줄기류': ['죽순', '도라지', '더덕']
}


# 분류 함수 정의
def categorize_item(item):
    for category, items in category_mapping.items():
        if item in items:
            return category
    return 'None'  # 해당되지 않는 경우

# '임산물분류' 열 생성
data_nongsan['임산물분류'] = data_nongsan['임산물명'].apply(categorize_item)
data_nongsan

### 농산물 데이터, 임산물 데이터 합치기 - 농산물 데이터 전체와 임산물 데이터'임산물분류'의 각 카테고리별로 반씩 사용

In [ ]:
# 농산물 데이터랑 컬럼명 일치시키기
data_imsan.rename(columns={'추출단어명': '임산물명'}, inplace=True)

In [ ]:
# 분류 기준 딕셔너리 생성
category_mapping = {
    '견과류': ['호두', '밤', '도토리', '잣', '개암', '은행'],
    '버섯류': ['표고버섯', '꽃송이버섯', '송이버섯', '석이버섯', '싸리버섯', '능이버섯', '목이버섯', '복령버섯'],
    '나물류': ['산마늘', '고사리', '눈개승마', '곤드레', '고려엉겅퀴', '참나물', '어수리', '두릅', '취나물', '고비', '원추리', '곰취', '들메나무순'],
    '열매과일류': ['대추', '머루', '복분자', '산딸기', '돌배', '감', '석류', '다래'],
    '뿌리줄기류': ['죽순', '도라지', '더덕']
}


# 분류 함수 정의
def categorize_item(item):
    for category, items in category_mapping.items():
        if item in items:
            return category
    return 'None'  # 해당되지 않는 경우

# '임산물분류' 열 생성
data_imsan['임산물분류'] = data_imsan['임산물명'].apply(categorize_item)

In [ ]:
import pandas as pd

# 각 임산물분류 별로 데이터를 반씩 나누어 저장
data_imsan_use = pd.DataFrame()
data_imsan_unuse = pd.DataFrame()

# 각 분류에 대해 데이터를 반씩 나누어 저장
for category in data_imsan['임산물분류'].unique():
    category_data = data_imsan[data_imsan['임산물분류'] == category]
    category_data_sample = category_data.sample(frac=0.5, random_state=42)
    data_imsan_use = pd.concat([data_imsan_use, category_data_sample])
    data_imsan_unuse = pd.concat([data_imsan_unuse, category_data.drop(category_data_sample.index)])

# 각각의 데이터프레임을 csv 파일로 저장
data_imsan_use.to_csv('data_imsan_use.csv', index=False, encoding='utf-8-sig')
data_imsan_unuse.to_csv('data_imsan_unuse.csv', index=False, encoding='utf-8-sig')

print("임산물 분류 데이터가 data_imsan_use.csv와 data_imsan_unuse.csv에 저장되었습니다.")


In [ ]:
data = pd.concat([data_imsan_use, data_nongsan], axis=0, ignore_index=True)
data

In [ ]:
data['임산물분류'].value_counts()

### 예측에 필요한 데이터프레임으로 만들기
- nan 값은 없으나 false 값 아직 많이 포함됨

In [ ]:
# 정보획득일자 -> 연도, 월 로 나누기
data['연도'] = data['정보획득일자'].astype(str).str[:4]
data['월'] = data['정보획득일자'].astype(str).str[4:6]

In [ ]:
# NaN 값을 'false'로 채우기
data['이용자직업명'] = data['이용자직업명'].fillna('false')
data['가구소득그룹명'] = data['가구소득그룹명'].fillna('false')
data['주거형태명'] = data['주거형태명'].fillna('false')
data['자동차그룹명'] = data['자동차그룹명'].fillna('false')

# 결측치 및 불필요한 공백을 'false'로 채우기
data.loc[data['결혼여부명'] == '      ', '결혼여부명'] = 'false'
data.loc[data['자녀여부명'] == '          ', '자녀여부명'] = 'false'
data.loc[data['자동차그룹명'].str.strip() == '', '자동차그룹명'] = 'false'

# '자동차그룹명'의 연속된 공백 제거
data['자동차그룹명'] = data['자동차그룹명'].str.replace(r'\s{3,}', '', regex=True)

In [ ]:
data.isnull().sum()

In [ ]:
# 예측에 필요한 컬럼만 빼서 데이터프레임으로 만들기
modeling_col = ['임산물분류','임산물명','연도','월','이용자성별코드','연령그룹코드','이용자직업명','가구소득그룹명','결혼여부명','자녀여부명','주거형태명',
      '이용자지역시도명','이용자지역구명','이용자지역동명']
modeling_data = data[modeling_col]
modeling_data

In [ ]:
# 카테고리형으로 모든 컬럼 변환
for col in modeling_col:
    modeling_data.loc[:, col] = modeling_data[col].astype('category')

### 임산물별 예측 모델

In [ ]:
#imsan_type = ['대추', '호두', '머루', '복분자', '산마늘', '산딸기', '죽순', '돌배', '고사리', '표고버섯',
#            '도라지', '감', '눈개승마', '곤드레', '꽃송이버섯', '밤', '도토리', '고려엉겅퀴', '송이버섯',
#            '더덕', '참나물', '어수리', '두릅', '석이버섯', '석류', '싸리버섯', '잣', '능이버섯',
#            '목이버섯', '복령버섯', '취나물', '개암', '다래', '고비', '원추리', '은행', '곰취',
#            '들메나무순']

In [ ]:
nongsan_type = ['견과류', '버섯류', '나물류', '열매과일류', '뿌리줄기류']

In [ ]:
# '임산물구매' 컬럼 생성 후 'O'와 'X'로 값 설정 및 범주형 변환
modeling_data['임산물구매'] = modeling_data['임산물분류'].apply(lambda x: 'X' if x == 'None' else 'O').astype('category')

In [ ]:
modeling_data['임산물분류'].value_counts()

In [ ]:
modeling_data['임산물구매'].value_counts()

In [ ]:
modeling_data = modeling_data.sample(frac=1, random_state=42).reset_index(drop=True)
modeling_data

### 다중공선성 확인

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 상관관계 행렬 계산
df = modeling_data[['임산물명', '연도', '월', '이용자성별코드', '연령그룹코드', '이용자직업명',
                    '가구소득그룹명', '결혼여부명', '자녀여부명', '주거형태명',
                    '이용자지역시도명', '이용자지역구명', '이용자지역동명']].apply(lambda x: pd.factorize(x)[0])
corr_matrix = df.corr()

# 상관계수가 0.8 이상인 변수 쌍 추출
high_corr_pairs = []
for col1 in corr_matrix.columns:
    for col2 in corr_matrix.columns:
        if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > 0.7:
            high_corr_pairs.append((col1, col2, corr_matrix.loc[col1, col2]))

print(high_corr_pairs)

## Logistic Regression

In [ ]:
import os
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import pandas as pd

# 모델 저장할 디렉토리 설정
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류'] == imsan]
    onetype_length = len(imsan_onetype)

    # None 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])
    onetype_none_data = onetype_none_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # X, y 나누기
    X = onetype_none_data[['연도', '월', '이용자성별코드', '연령그룹코드', '이용자직업명', '가구소득그룹명', '결혼여부명', '자녀여부명', '주거형태명',
                           '이용자지역시도명', '이용자지역구명', '이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Logistic Regression 모델 정의
    model = LogisticRegression(max_iter=100, random_state=42)

    # 모델 학습
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rae = sum(abs(y_test - y_pred)) / sum(abs(y_test - y_test.mean()))
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_LogisticRegression_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

# 성능 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
print("\nLogistic Regression")
print(results_df)

# results_df를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_logistic_regression.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")



## Random forest

In [ ]:
# 모델 저장할 디렉토리 설정
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류'] == imsan]
    onetype_length = len(imsan_onetype)

    # None 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])
    onetype_none_data = onetype_none_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # X, y 나누기
    X = onetype_none_data[['연도', '월', '이용자성별코드', '연령그룹코드', '이용자직업명', '가구소득그룹명', '결혼여부명', '자녀여부명', '주거형태명',
                           '이용자지역시도명', '이용자지역구명', '이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Random Forest 모델 정의
    model = RandomForestClassifier(n_estimators=1000, random_state=42)

    # 모델 학습
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rae = sum(abs(y_test - y_pred)) / sum(abs(y_test - y_test.mean()))
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_RandomForest_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

# 성능 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
print("\nRandom Forest")
print(results_df)

# results_df를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_RandomForest.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")

### XGBoost

In [ ]:
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류']==imsan]
    onetype_length = len(imsan_onetype)

    # 해당 데이터셋만큼 임산물명이 None인 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])
    onetype_none_data.sample(frac=1, random_state=42).reset_index(drop=True)
    #print(onetype_none_data['임산물구매'].value_counts())

    # X, y 나누기
    X = onetype_none_data[['연도','월','이용자성별코드','연령그룹코드','이용자직업명','가구소득그룹명','결혼여부명', '자녀여부명','주거형태명',
      '이용자지역시도명','이용자지역구명','이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 ##############
    model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    random_state=42
    )

    # 모델 학습
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rae = sum(abs(y_test - y_pred)) / sum(abs(y_test - y_test.mean()))
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_XGBoost_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

# 성능 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
print("\nXGBoost")
print(results_df)

# results_df를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_XGBoost.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")

### NGBoost

In [ ]:
# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type[:]:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류']==imsan]
    onetype_length = len(imsan_onetype)

    # 해당 데이터셋만큼 임산물명이 None인 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])

    # X, y 나누기
    X = onetype_none_data[['연도','월','이용자성별코드','연령그룹코드','이용자직업명','가구소득그룹명','결혼여부명','자녀여부명','주거형태명',
      '이용자지역시도명','이용자지역구명','이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 ##############
    model = NGBClassifier(random_state=42, n_estimators=1000)

    # 모델 학습
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rae = sum(abs(y_test - y_pred)) / sum(abs(y_test - y_test.mean()))
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_NGBoost_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

results_df = pd.DataFrame(results)
print("nXGBoost")
print(results_df)
print("----")
print("----")
print("----")
print("----")
# results_df를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_NGBoost.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")

### N-BEATS

In [ ]:
# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type[:]:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류']==imsan]
    onetype_length = len(imsan_onetype)

    # 해당 데이터셋만큼 임산물명이 None인 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])

    # X, y 나누기
    X = onetype_none_data[['연도','월','이용자성별코드','연령그룹코드','이용자직업명','가구소득그룹명','결혼여부명','자녀여부명','주거형태명',
      '이용자지역시도명','이용자지역구명','이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    # y_train_tensor = torch.tensor(y_train_encoded.values, dtype=torch.float32).view(-1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    # y_test_tensor = torch.tensor(y_test_encoded.values, dtype=torch.float32).view(-1)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # 모델 ##############
    model = NBeatsNet(
        forecast_length=1,
        backcast_length=X_train_tensor.shape[1],
        stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
        nb_blocks_per_stack=3,
        thetas_dim=(4, 4),
        share_weights_in_stack=False,
        hidden_layer_units=512,
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = torch.nn.MSELoss()

    # 모델 학습
    n_epochs = 50
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        backcast, forecast = model(X_train_tensor)
        predictions = forecast.view(-1)

        # Convert target tensors to float32 for MSELoss
        y_train_tensor_float = y_train_tensor.float()

        loss = loss_fn(predictions, y_train_tensor_float)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    model.eval()
    with torch.no_grad():
        _, y_pred_tensor = model(X_test_tensor)
    y_pred = (y_pred_tensor.numpy() > 0.5).astype(int)
    #y_pred_labels = label_encoder.inverse_transform(y_pred)

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    # rae = sum(abs(y_test - y_pred)) / sum(abs(y_test - y_test.mean()))
    def relative_absolute_error(y_true, y_pred):
        # Flatten the arrays if they are multi-dimensional
        y_true = np.ravel(y_true)
        y_pred = np.ravel(y_pred)

        # Calculate RAE
        rae = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true - np.mean(y_true)))

        return rae
    rae_value = relative_absolute_error(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae_value}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_N-BEATS_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

results_df = pd.DataFrame(results)
print("nXGBoost")
print(results_df)
# results_df를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_N-BEATS.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")

## MLP

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import os
import pickle

# 결과를 저장할 리스트 초기화
results = []

# 임산물명이 None인 데이터
none_data = modeling_data[modeling_data['임산물구매'] == 'X']

# 임산물별 예측 모델
for imsan in nongsan_type[:]:
    print(imsan)

    # 해당 임산물에 해당하는 데이터셋
    imsan_onetype = modeling_data[modeling_data['임산물분류'] == imsan]
    onetype_length = len(imsan_onetype)

    # 해당 데이터셋만큼 임산물명이 None인 데이터를 undersampling
    undersampled_none_data = none_data.sample(n=onetype_length, random_state=42)

    # 해당 임산물 데이터셋 + None 언더샘플링 데이터셋
    onetype_none_data = pd.concat([undersampled_none_data, imsan_onetype])

    # X, y 나누기
    X = onetype_none_data[['연도', '월', '이용자성별코드', '연령그룹코드', '이용자직업명', '가구소득그룹명', '결혼여부명',
                          '자녀여부명', '주거형태명', '이용자지역시도명', '이용자지역구명', '이용자지역동명']]
    y = onetype_none_data['임산물구매']

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # y를 LabelEncoder로 숫자형으로 변환
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y)

    # 학습용, 테스트용 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # MLP 모델 정의
    class MLP(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim):
            super(MLP, self).__init__()
            self.layers = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, output_dim)
            )

        def forward(self, x):
            return self.layers(x)

    # 모델 초기화
    input_dim = X_train_tensor.shape[1]
    hidden_dim = 512  # 은닉층 유닛 수
    output_dim = len(np.unique(y))  # 분류 클래스 수
    model = MLP(input_dim, hidden_dim, output_dim)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    # 모델 학습
    n_epochs = 1000
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = loss_fn(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    # 테스트 데이터로 성능 평가
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = y_pred_tensor.argmax(dim=1).numpy()  # 예측 클래스 선택

    # 성능 계산
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    def relative_absolute_error(y_true, y_pred):
        y_true = np.ravel(y_true)
        y_pred = np.ravel(y_pred)
        rae = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true - np.mean(y_true)))
        return rae

    rae_value = relative_absolute_error(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'RAE: {rae_value}')

    # 결과 저장
    results.append({
        'location_type': imsan,
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RAE': rae_value
    })

    # 모델 저장 (분류명_모델명 형식으로 저장)
    model_filename = os.path.join(model_dir, f"{imsan}_MLP_model.pkl")
    with open(model_filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {model_filename}")

results_df = pd.DataFrame(results)
print("MLP Results")
print(results_df)

# 결과를 txt 파일로 저장
results_txt_filename = os.path.join(model_dir, "results_MLP.txt")
with open(results_txt_filename, 'w') as file:
    file.write(results_df.to_string(index=False))

print(f"Results saved as {results_txt_filename}")


## 임산물 데이터에서 모델 학습에 사용하지 않은 나머지 데이터로 예측


* 견과류: XGBoost
* 버섯류: Logistic Regression
* 나물류: XGBoost
* 열매과일류:  XGBoost
* 뿌리줄기류: RandomForest


In [ ]:
# data_imsan_unuse 데이터 불러오기
data_imsan_unuse = pd.read_csv("data_imsan_unuse.csv")
data_imsan_test = data_imsan_unuse

In [ ]:
# 정보획득일자 -> 연도, 월 로 나누기
data_imsan_test['연도'] = data_imsan_test['정보획득일자'].astype(str).str[:4]
data_imsan_test['월'] = data_imsan_test['정보획득일자'].astype(str).str[4:6]

# NaN 값을 'false'로 채우기
data_imsan_test['이용자직업명'] = data_imsan_test['이용자직업명'].fillna('false')
data_imsan_test['가구소득그룹명'] = data_imsan_test['가구소득그룹명'].fillna('false')
data_imsan_test['주거형태명'] = data_imsan_test['주거형태명'].fillna('false')
data_imsan_test['자동차그룹명'] = data_imsan_test['자동차그룹명'].fillna('false')

# 결측치 및 불필요한 공백을 'false'로 채우기
data_imsan_test.loc[data_imsan_test['결혼여부명'] == '      ', '결혼여부명'] = 'false'
data_imsan_test.loc[data_imsan_test['자녀여부명'] == '          ', '자녀여부명'] = 'false'
data_imsan_test.loc[data_imsan_test['자동차그룹명'].str.strip() == '', '자동차그룹명'] = 'false'

# '자동차그룹명'의 연속된 공백 제거
data_imsan_test['자동차그룹명'] = data_imsan_test['자동차그룹명'].str.replace(r'\s{3,}', '', regex=True)

# 예측에 필요한 컬럼만 빼서 데이터프레임으로 만들기
test_data = data_imsan_test.copy()
test_data

# 카테고리형으로 모든 컬럼 변환
for col in test_data:
    test_data.loc[:, col] = test_data[col].astype('category')

In [ ]:
test_data

In [ ]:
test_data.info()

In [ ]:
test_data['임산물분류'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import pickle

# 모델 경로 설정
model_dir = "models"

# 각 임산물분류별로 지정된 모델
model_mapping = {
    '견과류': 'XGBoost',
    '버섯류': 'XGBoost',
    '나물류': 'XGBoost',
    '열매과일류': 'XGBoost',
    '뿌리줄기류': 'XGBoost'
}

# 결과 저장할 리스트 초기화
predictions = []

# 각 임산물분류별로 모델을 적용하여 예측 수행
for category, model_name in model_mapping.items():
    # 해당 임산물분류에 해당하는 데이터 필터링
    category_data = test_data[test_data['임산물분류'] == category]

    # 예측에 사용할 피처 선택
    X = category_data[['연도', '월', '이용자성별코드', '연령그룹코드', '이용자직업명', '가구소득그룹명', '결혼여부명',
                      '자녀여부명', '주거형태명', '이용자지역시도명', '이용자지역구명', '이용자지역동명']].copy()

    # X의 각 범주형 변수를 LabelEncoder로 인코딩
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    # 모델 파일 로드
    model_filename = os.path.join(model_dir, f"{category}_{model_name}_model.pkl")
    with open(model_filename, 'rb') as file:
        model = pickle.load(file)

    # 예측 수행
    y_pred = model.predict(X)

    # 예측 결과 저장
    category_predictions = category_data.copy()
    category_predictions['예측값'] = y_pred
    predictions.append(category_predictions)

# 예측 결과를 하나의 데이터프레임으로 결합
predictions_df = pd.concat(predictions, ignore_index=True)

# 예측 결과를 CSV 파일로 저장
predictions_csv_filename = "imsan_predictions.csv"
predictions_df.to_csv(predictions_csv_filename, index=False, encoding='utf-8-sig')
